In [1]:
import sys
import os
import math
import numpy as np

states = { "s": 0, "E": 1, "5": 2, "I" : 3, "e": 4}
id2state = {0: "s", 1: "E", 2: "5", 3: "I", 4: "e"}

state_transition_prob = np.array([[0.0, 1.0, 0.0, 0.0, 0.0], 
                                  [0.0, 0.9, 0.1, 0.0, 0.0], 
                                  [0.0, 0.0, 0.0, 1.0, 0.0],
                                  [0.0, 0.0, 0.0, 0.9, 0.1],
                                  [0.0, 0.0, 0.0, 0.0, 0.0]]) 
emission_nuc_codes = {'A': 0, 
                      'C': 1, 
                      'G': 2, 
                      'T': 3}

emission_probs = np.array([[0.00, 0.00, 0.00, 0.00], 
                           [0.25, 0.25, 0.25, 0.25],
                           [0.05, 0.00, 0.95, 0.00],
                           [0.40, 0.10, 0.10, 0.40],
                           [0.00, 0.00, 0.00, 0.00]]) 

query_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"


In [2]:
def get_log_prob_for_state_path (state_path, query_sequence):
    res = math.log(0.25)
    for i in range(1, len(state_path)):
        res += math.log(state_transition_prob[ states[state_path[i-1]] ][ states[state_path[i]] ]*emission_probs[ states[state_path[i]] ][ emission_nuc_codes[query_sequence[i]] ])
    return res

In [3]:
# CTTCATGTGAAAGCAGACGTAAGTCA 
# EEEEEE5IIIIIIIIIIIIIIIIIII
k1 = get_log_prob_for_state_path("EEEEEE5IIIIIIIIIIIIIIIIIII", "CTTCATGTGAAAGCAGACGTAAGTCA") +  math.log (0.1)
print (k1)


-43.89740030179307


In [ ]:
# CTTCATGTGAAAGCAGACGTAAGTCA 
# EEEEEEEE5IIIIIIIIIIIIIIIII
k2 = get_log_prob_for_state_path("EEEEEEEE5IIIIIIIIIIIIIIIII", "CTTCATGTGAAAGCAGACGTAAGTCA") + math.log (0.1)
print (k2)


In [ ]:
# CTTCATGTGAAAGCAGACGTAAGTCA 
# EEEEEEEEEEEE5IIIIIIIIIIIII
k3 = get_log_prob_for_state_path("EEEEEEEEEEEE5IIIIIIIIIIIII", "CTTCATGTGAAAGCAGACGTAAGTCA") + math.log (0.1)
print (k3)


In [ ]:
# CTTCATGTGAAAGCAGACGTAAGTCA 
# EEEEEEEEEEEEEEE5IIIIIIIIII
k4 = get_log_prob_for_state_path("EEEEEEEEEEEEEEE5IIIIIIIIII", "CTTCATGTGAAAGCAGACGTAAGTCA") + math.log (0.1)
print (k4)


In [ ]:
# CTTCATGTGAAAGCAGACGTAAGTCA 
# EEEEEEEEEEEEEEEEEE5IIIIIII
k5 = get_log_prob_for_state_path("EEEEEEEEEEEEEEEEEE5IIIIIII", "CTTCATGTGAAAGCAGACGTAAGTCA") + math.log (0.1)
print (k5)


In [ ]:
# CTTCATGTGAAAGCAGACGTAAGTCA 
# EEEEEEEEEEEEEEEEEEEEEE5III
k6 = get_log_prob_for_state_path("EEEEEEEEEEEEEEEEEEEEEE5III", "CTTCATGTGAAAGCAGACGTAAGTCA") + math.log (0.1)
print (k6)


In [ ]:
# CTTCATGTGAAAGCAGACGTAAGTCA 
# EEEEEEEEEEEEEEEEEEEEEEEEEE
only_E = get_log_prob_for_state_path("EEEEEEEEEEEEEEEEEEEEEEEEEE", "CTTCATGTGAAAGCAGACGTAAGTCA") + math.log (0.1)
print (only_E)


### Design of the Viterbi Value matrix

Rows correspond to the hidden states, and the columns correspond to the emissions that is the observed nucleotide sequences. Here I am showing the calculation for the first two nucletides. 

```
             C                                                          T     T
s [s-s-C(0.00) max(s-s-C-s-T, s-E-C-s-T, s-5-C-s-T, s-I-C-s-T, s-e-C-s-T)     .] 
E [s-E-C(0.25) max(s-s-C-E-T, s-E-C-E-T, s-5-C-E-T, s-I-C-E-T, s-e-C-E-T)     .] 
5 [s-5-C(0.00) max(s-s-C-5-T, s-E-C-5-T, s-5-C-5-T, s-I-C-5-T, s-e-C-5-T)     .]
I [s-I-C(0.00) max(s-s-C-I-T, s-E-C-I-T, s-5-C-I-T, s-I-C-I-T  s-e-C-I-T)     .]
e [s-e-C(0.00) max(s-s-C-e-T, s-E-C-e-T, s-5-C-e-T, s-I-C-e-T, s-e-C-e-T)     .]

```

It is important to remember that you will be working in the log scale.

In [ ]:
# Initiate two matrices: 
# viterbi_value_matrix: to store the values described in the documentation above 
# viterbi_trace_matrix: to store the path the lead to the the maximum value in each cell
# For example, the first column of viterbi_trace_matrix will be 
# [0] indicating start state released `C`: even though not possible - but we just initiate
# [1] indicating Exon state released `C`:
# [2] indicating 5'ss state released `C`: even though not possible - but we just initiate
# [3] indicating Intron state released `C`: 
# [4] indicating end state released `C`: even though not possible - but we just initiate

### Implementation of Viterbi Algorithm
Write a function `calculate_prob_for_a_node()` that populate a single cell in the matrix. The function will return two values:
1. the maximum value, for example, look at the 2nd row, 2nd column in the matrix: `max(s-s-C-E-T, s-E-C-E-T, s-5-C-E-T, s-I-C-E-T, s-e-C-E-T)`. If the probability for `s-E-C-E-T` is highest (lets say X), then the function should return `X`

**AND** 

2. The index of that maximum value described in the first point: so index of X is `1` (recall that Python works on the 0-based index system)

- Populate `viterbi_value_matrix` with `X` for row 2 and col 2

- Populate `viterbi_trace_matrix` with `1` for row 2 and col 2

In [ ]:
# Write for loops to iterate over the whole Viterbi Value matrix. 
# Each time, call the function 

In [ ]:
# Write a function to trace the state path that gave the maximum probability. 
# This will be the final result. 


# HINT: You should first find the maximum value in the last column of `viterbi_value_matrix`,
# because that is the one with the largest probability. 
# The index of that value is the state of the last nucleotide.  